In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install SentencePiece
!pip install 'accelerate>=0.26.0'
!pip install transformers[torch]
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 587.1 kB/s eta 0:00:00a 0:00:01
zsh:1: no matches found: transformers[torch]


In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch
import sacrebleu
from sklearn.model_selection import train_test_split

# Load CSV file
data_df = pd.read_csv("train.csv")  # Update with your CSV file name
data_df = data_df.dropna(subset=["text", "gloss"]).reset_index(drop=True)

# Split into training and testing (80:20)
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Use T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")



In [19]:
# Prepare dataset with task prefix
def tokenize_function(examples):
    inputs = ["translate English to Gloss: " + sentence for sentence in examples["text"]]
    targets = examples["gloss"]

    model_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding='max_length', truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create Dataset objects from train and test DataFrames
train_dataset = Dataset.from_pandas(train_df[["text", "gloss"]])
test_dataset = Dataset.from_pandas(test_df[["text", "gloss"]])

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/70168 [00:00<?, ? examples/s]

/opt/anaconda3/envs/dsan5400/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/17542 [00:00<?, ? examples/s]

In [20]:
# Load model (T5 for translation)
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

# Evaluate model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

/opt/anaconda3/envs/dsan5400/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/gr/z7mvrtxx0950xmcc6nx7xxhr0000gn/T/ipykernel_30740/4085395391.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.267300,0.162340
2,0.185700,0.110463
3,0.152000,0.089953
4,0.139000,0.080833
5,0.132500,0.078281


Evaluation Results: {'eval_loss': 0.07828138768672943, 'eval_runtime': 256.0411, 'eval_samples_per_second': 68.512, 'eval_steps_per_second': 4.284, 'epoch': 5.0}


In [24]:
# BLEU Score Evaluation
def compute_bleu(model, tokenizer, dataset):
    references = []
    predictions = []

    model.eval()
    device = torch.device("cpu")  # Switch to CPU to avoid MPS issues
    model.to(device)

    for example in dataset:
        input_text = "translate English to Gloss: " + example["text"]
        input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=128).input_ids.to(device)

        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=128)

        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        references.append([example["gloss"]])
        predictions.append(prediction)

    bleu = sacrebleu.corpus_bleu(predictions, references)
    return bleu.score


# BLEU on test set
raw_test = test_df.to_dict(orient="records")
bleu_score = compute_bleu(model, tokenizer, raw_test)
print("BLEU Score on Test Set:", bleu_score)


BLEU Score on Test Set: 41.11336169005198
